<a href="https://colab.research.google.com/github/Vdharmaram/Neuaral-Network_Final-exam/blob/main/New_Exam2_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, SGD
import itertools


In [6]:

(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# One-hot encode
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)


In [7]:

def build_model(optimizer='adam'):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(2, 2),
        Dropout(0.3),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(2, 2),
        Dropout(0.3),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:

learning_rates = [0.001, 0.0005]
batch_sizes = [64, 128]
optimizers = {'adam': Adam, 'sgd': SGD}

best_val_acc = 0
best_history = None
best_model = None

for lr in learning_rates:
    for bs in batch_sizes:
        for name, opt_class in optimizers.items():
            print(f"Training with {name}, lr={lr}, batch_size={bs}")
            model = build_model(opt_class(learning_rate=lr))
            rlrop = ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.5, verbose=1)
            history = model.fit(x_train, y_train_cat,
                                epochs=15,
                                batch_size=bs,
                                validation_split=0.2,
                                callbacks=[rlrop],
                                verbose=0)
            val_acc = history.history['val_accuracy'][-1]
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                best_history = history
                best_model = model


Training with adam, lr=0.001, batch_size=64


In [ ]:

y_pred = best_model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test, y_pred_classes)

plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()


In [ ]:

history = best_history

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title('Accuracy over epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:

fig, axes = plt.subplots(1, 5, figsize=(15, 4))
for i, ax in enumerate(axes):
    ax.imshow(x_test[i].squeeze(), cmap='gray')
    ax.set_title(f"True: {y_test[i]}, Pred: {y_pred_classes[i]}")
    ax.axis('off')
plt.suptitle("Sample Predictions")
plt.show()


In [ ]:

from collections import defaultdict

class_correct = defaultdict(int)
class_total = defaultdict(int)

for i in range(len(y_test)):
    label = y_test[i]
    pred = y_pred_classes[i]
    class_total[label] += 1
    if label == pred:
        class_correct[label] += 1

labels = [f"{i}" for i in range(10)]
accuracy_per_class = [class_correct[i] / class_total[i] for i in range(10)]

plt.figure(figsize=(10, 6))
sns.barplot(x=labels, y=accuracy_per_class)
plt.title("Accuracy per Class")
plt.xlabel("Class Label")
plt.ylabel("Accuracy")
plt.ylim(0, 1)
plt.show()
